In [4]:
import os
import cv2     # for capturing videos
import math 
import geocoder
import requests
%matplotlib inline
import pandas as pd
from twilio.rest import Client
from geopy.geocoders import Nominatim
from keras.preprocessing import image   # for preprocessing the images
import numpy as np    # for mathematical operations
from keras.utils import np_utils
from matplotlib import pyplot as plt 
from skimage.transform import resize   # for resizing images

In [ ]:
count = 0
videoFile = "Accidents.mp4"
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

In [ ]:
img = plt.imread('0.jpg')   # reading image using its name
plt.imshow(img)

In [4]:
data = pd.read_csv('mapping.csv')     # reading the csv file
data.head()

,Image_ID,Class
0,0.jpg,1
1,1.jpg,1
2,2.jpg,1
3,3.jpg,1
4,4.jpg,1


In [5]:
X = [ ]     # creating an empty array
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)  # storing each image in array X
X = np.array(X)    # converting list to array

In [6]:
y = data.Class
dummy_y = np_utils.to_categorical(y)

In [7]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

In [32]:
from keras.applications.vgg19 import preprocess_input
X = preprocess_input(X,data_format=None)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)

In [34]:
from keras.models import Sequential
from keras.applications.vgg19 import VGG19
from keras.layers import Dense, InputLayer, Dropout

In [35]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3)) 

In [36]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_train.shape, X_valid.shape

3/3 [==============================] - 12s 3s/step


((155, 7, 7, 512), (67, 7, 7, 512))

In [37]:
X_train = X_train.reshape(155, 7*7*512)      # converting to 1-D
X_valid = X_valid.reshape(67, 7*7*512)

In [38]:
train = X_train/X_train.max()      # centering the data
X_valid = X_valid/X_train.max()

In [39]:
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid')) # hidden layer
model.add(Dense(2, activation='softmax'))    # output layer

In [40]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 1024)              25691136  
                                                                 
 dense_3 (Dense)             (None, 2)                 2050      
                                                                 
Total params: 25,693,186
Trainable params: 25,693,186
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [42]:
model.fit(train, y_train, epochs=10, validation_data=(X_valid, y_valid))

Epoch 1/10
5/5 [==============================] - 3s 416ms/step - loss: 1.1314 - accuracy: 0.4710 - val_loss: 0.7502 - val_accuracy: 0.5970
Epoch 2/10
5/5 [==============================] - 2s 387ms/step - loss: 0.5344 - accuracy: 0.7484 - val_loss: 0.9762 - val_accuracy: 0.6119
Epoch 3/10
5/5 [==============================] - 2s 401ms/step - loss: 0.2948 - accuracy: 0.8774 - val_loss: 1.0181 - val_accuracy: 0.6866
Epoch 4/10
5/5 [==============================] - 2s 347ms/step - loss: 0.2389 - accuracy: 0.8968 - val_loss: 0.8410 - val_accuracy: 0.6567
Epoch 5/10
5/5 [==============================] - 2s 354ms/step - loss: 0.1300 - accuracy: 0.9613 - val_loss: 0.8254 - val_accuracy: 0.6716
Epoch 6/10
5/5 [==============================] - 2s 360ms/step - loss: 0.1014 - accuracy: 0.9742 - val_loss: 0.7679 - val_accuracy: 0.8060
Epoch 7/10
5/5 [==============================] - 2s 352ms/step - loss: 0.0691 - accuracy: 0.9935 - val_loss: 0.7766 - val_accuracy: 0.7910
Epoch 8/10
5/5 [====

In [43]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
count = 0
videoFile = "Accident-1.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [45]:
test = pd.read_csv('test.csv')

In [46]:
test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [47]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

In [48]:
# preprocessing the images
test_image = preprocess_input(test_image, data_format=None)

# extracting features from the images using pretrained model
test_image = base_model.predict(test_image)
test_image.shape

1/1 [==============================] - 1s 1s/step


(9, 7, 7, 512)

In [49]:
test_image = test_image.reshape(9, 7*7*512)

# zero centered images
test_image = test_image/test_image.max()

In [50]:
predictions = model.predict(test_image)

1/1 [==============================] - 0s 54ms/step


In [51]:
print(predictions)

[[0.03436817 0.96563184]
 [0.12758933 0.87241066]
 [0.09631952 0.90368044]
 [0.13595417 0.86404586]
 [0.16050208 0.839498  ]
 [0.29410216 0.70589787]
 [0.512804   0.48719606]
 [0.46626845 0.5337316 ]
 [0.47833917 0.52166086]]


In [52]:
for i in range (0,9):
    if predictions[i][0]<predictions[i][1]:
        print("No Accident")
    else:
        print("Accident")
    

No Accident
No Accident
No Accident
No Accident
No Accident
No Accident
Accident
No Accident
No Accident


In [53]:
geoLoc = Nominatim(user_agent="GetLoc")
g = geocoder.ip('me')
locname = geoLoc.reverse(g.latlng)
account_sid = "AC8d25ed394916e01596edd70b2abd77a1"
auth_token = "98bd4f96d016af71efbb0f21811d835d"
client = Client(account_sid, auth_token)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=28.6519&lon=77.2315&format=json&addressdetails=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))

In [54]:
cap = cv2.VideoCapture('Accident-1.mp4')
i=0
flag=0
while(True):
    ret,frame=cap.read()
    if ret==True:
        if predictions[int(i/15)%9][0]<predictions[int(i/15)%9][1]:
            predict="No Accident"
        else:
            predict="Accident"
            flag=1
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame,
                predict,
                (50, 50),
                font, 1,
                (0, 255, 255),
                3,
                cv2.LINE_4)
        cv2.imshow('Frame', frame)
        i=i+1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
if flag==1:
    client.messages.create(
                 body="Accident detected in "+locname.address,
                 from_= "+16502977583",
                 to= "+919654429598" )

# release the cap object
cap.release()
# close all windows
cv2.destroyAllWindows()


NameError: name 'client' is not defined